In [1]:
import xmltodict, json
import os
import ast
import pandas as pd
data_path = '/Users/jingweizhang/Documents/Projects/BoE_news_data/'
data_hist = 'text_en_BoE_199801_201710'
data_recent = 'text_en_BoE_201711_201908'

In [2]:
list_of_files_recent = [i for i in os.listdir(data_path + data_recent) if i.endswith('XML')]
list_of_files_hist = [i for i in os.listdir(data_path + data_hist) if i.endswith('XML')]

In [34]:
def get_altId(altid_item):
    for value in altid_item:
        if value['@type'] == "idType:USN":
            return value['#text']
        
def xml_to_json(file_name, data_dir, save_output = False):
    xml_file = open(data_path + data_dir + '/' + file_name).read()
    obj = xmltodict.parse(xml_file)
    dict_file = json.loads(json.dumps(obj))
    if save_output == True:
        with open(data_path + 'json_data/'+ file_name[:-3] + '.json', 'w') as outfile:
             json.dump(dict_file, outfile)
    return dict_file

def json_to_dict_2017(file_name,data_dir):
    dict_file = xml_to_json(file_name,data_dir)
    news_item = dict_file['newsMessage']['itemSet']['newsItem']         
    news_dict = dict()
    
    news_dict = {'Id': file_name.split('_')[1][:-4],
                 'time': dict_file['newsMessage']['header']['sent'],
                 'language':news_item['contentMeta']['language']['@tag'],
                 'headline':news_item['contentMeta']['headline'],
                 'wordcount': int(news_item['contentSet']['inlineXML']['@wordcount']),
                 'genre': None,
                 'slugline': news_item['contentMeta']['slugline']['#text'],
                 'subject':','.join([value['@qcode'] for value in news_item['contentMeta']['subject'] if '@qcode' in value.keys()]),
                 'urgency': news_item['contentMeta']['urgency'],
                 'altId': get_altId(news_item['contentMeta']['altId'])
                 }
    if 'genre' in news_item['contentMeta'].keys():
        genres = news_item['contentMeta']['genre']
        if type(genres) == list:
            if 
            news_dict['genre'] = ','.join([value['name'] for value in genres])
        else:
            news_dict['genre'] = genres['name'] 
    
    return news_dict  

In [32]:
def json_to_dict_1998(file_name,data_dir):
    dict_file = xml_to_json(file_name,data_dir)
    news_item = dict_file['n-docbody']['document']
    news_dict = dict()
    
    ner_item = news_item['indexing']
    
    if type(ner_item) == list:
        ner_item = ner_item[1]
        
    try:
        ric_codes_list = news_item['indexing'][0]['business-terms']['ric-code-wrap']['ric-code']
        ric_codes = [value['#text'] for value in ric_codes_list]
    except:
        ric_codes = None
        
        
    def get_feature_list(classification_feature):
        '''
        feature: location, subject, industry
        
        '''
        if 'classification-terms' in ner_item.keys():
            block = 'pres-%s-block'%(classification_feature)
            wrap =  'pres-%s-wrap'%(classification_feature)
            feature_list = None
            if type(ner_item['classification-terms']) == list:
                if block in ner_item['classification-terms'][0].keys():
                    feature_list = ner_item['classification-terms'][0][block][wrap]
                elif block in ner_item['classification-terms'][1].keys():
                    feature_list = ner_item['classification-terms'][1][block][wrap]

            elif block in ner_item['classification-terms'].keys():
                 feature_list = ner_item['classification-terms'][block][wrap]

            if feature_list != None:
                if type(feature_list) == list:
                    feature_items = [value['pres-%s'%(classification_feature)] for value in feature_list]
                else:
                    feature_items = feature_list['pres-%s'%(classification_feature)]

                return ','.join(feature_items)

    news_dict = {'Id':file_name.split('.')[0], 
                'time': news_item['pub-info']['pub-date']['sort-pub-date'],
                 'language': news_item['@load-language'],
                 'headline': news_item['title-info']['title']['sort-title'],
                 'wordcount': int(news_item['content']['derived-word-count']['#text'].split()[-1]),
                 'location':get_feature_list('location'),
                 'subject':get_feature_list('subject'),
                 'industry':get_feature_list('industry'),
                 'ric_codes': ric_codes
                 }
    
    if type(news_dict['headline']) == dict:
        news_dict['headline'] = news_dict['headline']['#text']
    
    return news_dict

In [35]:
list_of_meta_recent = [json_to_dict_2017(file, data_recent) for file in list_of_files_recent]

In [33]:
list_of_meta_hist = [json_to_dict_1998(file, data_hist) for file in list_of_files_hist]

In [42]:
df_1998 = pd.DataFrame(list_of_meta_hist).sort_values(by = ['time'])
df_2017 = pd.DataFrame(list_of_meta_recent).sort_values(by = ['time'])

In [44]:
df_1998.to_csv('199801_201710.csv', index = False)
df_2017.to_csv('201711_201908.csv', index = False)

In [47]:
df_1998.groupby('time').Id.count().index

Index(['19980107', '19980108', '19980109', '19980113', '19980114', '19980115',
       '19980116', '19980211', '19980212', '19980213',
       ...
       '20170617', '20170802', '20170803', '20170804', '20170805', '20170913',
       '20170914', '20170915', '20170916', '20171101'],
      dtype='object', name='time', length=1884)